In [54]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import numpy as np

In [55]:
train = pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\train.csv")
test =  pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\test.csv")

In [56]:
# store target as Y
train=train[train.y<=170]
Y_train = train["y"]
#train.drop(["y"], axis=1, inplace=True)
id_test = test["ID"]
# Concat both sets
num_train = len(train)
all_data = pd.concat([train, test])

In [57]:
def replaceNonNumtoNum(c,train,all_data):
    g_NonNum=train.groupby(c)['y'].agg(['mean']).reset_index()
    g_NonNum=g_NonNum.sort_values(by=('mean'),ascending=False)
    g_NonNum['rnk']=g_NonNum['mean'].rank(ascending=False)
    all_data=pd.merge(all_data, g_NonNum, how='left', on=c)
    all_data[c]=all_data.rnk
    all_data.drop(['mean','rnk'], axis=1, inplace=True)
    return all_data

In [58]:
for f in all_data.columns:
    if all_data[f].dtype=='object':
        all_data=replaceNonNumtoNum(f,train,all_data)

In [59]:
all_data.drop(["y","ID"], axis=1, inplace=True)

In [60]:
X_train = all_data[:num_train]
X_test = all_data[num_train:]

In [61]:
X_train, X_cv, y_train, y_cv = train_test_split( X_train, Y_train, test_size = 0.1, random_state = 100)

In [62]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4,5,6]}

In [63]:
# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

In [64]:
grid = GridSearchCV(xgb, params)
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_child_weight': [4, 5], 'gamma': [0.3, 0.4, 0.5], 'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], 'max_depth': [2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [65]:
# Print the r2 score
print(r2_score(y_cv, grid.best_estimator_.predict(X_cv))) 

0.625069800804


In [66]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=5, missing=None,
       n_estimators=100, n_jobs=-1, nthread=-1, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=True, subsample=1.0)

In [67]:
# Save the file
y_test = grid.best_estimator_.predict(X_test)
submission = pd.DataFrame({
        "ID": id_test,
        "y": y_test
    })
submission.to_csv('xgb_second.csv', index=False) # LB : 0.55538 #


In [68]:
grid.best_score_

0.59629642932531646

In [1]:
grid.grid_scores_

NameError: name 'grid' is not defined

In [7]:
xgbtuned = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=5, missing=None,
       n_estimators=100, n_jobs=-1, nthread=-1, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=True, subsample=1.0)

C:\Users\user\Anaconda3\lib\site-packages\xgboost-0.6-py3.6.egg\xgboost\sklearn.py:171: DeprecationWarning: The nthread parameter is deprecated as of version .6.Please use n_jobs instead.nthread is deprecated.
  'nthread is deprecated.', DeprecationWarning)


In [8]:
xgbtuned.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=5, missing=None,
       n_estimators=100, n_jobs=-1, nthread=-1, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=True, subsample=1.0)

In [10]:
# Save the file
y_cv_pred = xgbtuned.predict(X_cv)

In [37]:
y_cv=pd.DataFrame(y_cv)
y_cv_pred=pd.DataFrame(y_cv_pred)
cv_set = pd.concat([y_cv,y_cv_pred,X_cv])

In [38]:
cv_set.to_csv('cv_set.csv',index=False)

In [41]:
y_cv_pred.to_csv('y_cv_pred.csv',index=False)

In [42]:
y_cv.to_csv('y_cv.csv',index=False)

In [43]:
X_cv.to_csv('X_cv.csv',index=False)